In [343]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from sympy.solvers import solve
from sympy import Symbol
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from scipy.optimize import curve_fit

Exercise 1

In [344]:
def Lorenz(lamb,sig,b,r):
    return lamb**3+(1+b+sig)*lamb**2+b*(sig+r)*lamb+2*sig*b*(r-1)

def zeros():
    x = Symbol('x')
    f = solve(Lorenz(x,sig,b,r),x)
    print("x_0 =", f[0])
    print("x_1 =", f[1])
    print("x_2 =", f[2])
    return f

In [346]:
sig = 10
b = 8/3
r = 0
lamb = np.linspace(-15,5,100)

x0 = zeros()

plt.grid()
plt.plot(lamb,Lorenz(lamb,sig,b,0))
plt.plot(lamb,Lorenz(lamb,sig,b,1))
plt.plot(lamb,Lorenz(lamb,sig,b,10))

plt.axhline(color="black", linewidth=1)
#print(intersection(lamb,Lorenz(lamb,sig,b,r),lamb,0))

x_0 = -10.7126694878696 + 0.e-22*I
x_1 = -4.15282623911189 + 0.e-20*I
x_2 = 1.19882906031485 + 0.e-20*I


Runge-Kutta-algorithm

In [347]:
def rk4_step(y0, t, f, h, f_args = {}):
    ''' Simple python implementation for one RK4 step. 
        Inputs:
            y_0    - M x 1 numpy array specifying all variables of the ODE at the current time step
            t      - current time step
            f      - function that calculates the derivates of all variables of the ODE
            h      - time step size
            f_args - Dictionary of additional arguments to be passed to the function f
        Output:
            yp1 - M x 1 numpy array of variables at time step t + h
            xp1 - time step t+h
    '''
    k1 = h * f(y0, t, **f_args)
    k2 = h * f(y0 + k1/2., t + h/2., **f_args)
    k3 = h * f(y0 + k2/2., t + h/2., **f_args)
    k4 = h * f(y0 + k3, t + h, **f_args)
    
    xp1 = t + h
    yp1 = y0 + 1./6.*(k1 + 2.*k2 + 2.*k3 + k4)
    
    return(yp1,xp1)

def rk4(y0, t, f, h, n, f_args = {}):
    ''' Simple implementation of RK4
        Inputs:
            y_0    - M x 1 numpy array specifying all variables of the ODE at the current time step
            t      - current time step
            f      - function that calculates the derivates of all variables of the ODE
            h      - time step size
            n      - number of steps
            f_args - Dictionary of additional arguments to be passed to the function f
        Output:
            yn - N+1 x M numpy array with the results of the integration for every time step (includes y0)
            xn - N+1 x 1 numpy array with the time step value (includes start t)
    '''
    yn = np.zeros((n+1, y0.shape[0]))
    xn = np.zeros(n+1)
    yn[0,:] = y0
    xn[0] = t
    
    for n in np.arange(1,n+1,1):
        yn[n,:], xn[n] = rk4_step(y0 = yn[n-1,:], t = xn[n-1], f = f, h = h, f_args = f_args)
    return(yn, xn)

# Be advised that the integration can take a while for large values of n (e.g >=10^5).


Exercise 2.1

In [348]:
#defining Lorenz-diff.eqs.
def x_dot(y,t):
    f = np.zeros(3)
    f[0] = -sig*(y[0]-y[1])
    f[1] = r*y[0]-y[1]-y[0]*y[2]
    f[2] = y[0]*y[1]-b*y[2]
    return f

In [349]:
#defining function to integrate lorenz diff.eqs. depending on values r,stepsize h and steps n
def lorenz_solver(r, h, n):

    #Initial values
    t = 0
    sig = 10
    b = 8/3
    

    #Initial values for the position at the fixpoints
    if r>1:
        a0 = np.sqrt(b*(r-1))
        y0 = np.array([a0,a0,r-1])+0.01
    else:
        y0 = np.array([0,0,0])+0.01

    #Computing the trajectories 
    x_vec,t=rk4(y0, t, x_dot, h, n)

    x = x_vec[:,0]
    y = x_vec[:,1]
    z = x_vec[:,2]
    
    print("finished")
    return x,y,z

In [352]:
r = 28
h = 0.01
n = 10000
x,y,z = lorenz_solver(r, h, n)

#Plot the result in x-y-plane
plt.plot(x,y)

finished


3D-Plot

In [353]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot3D(y,x,z)
ax.scatter3D(a0,a0,r-1,color="orange", label="fixed point")
ax.legend()
#ax.savefig("lorenz2.png")

In [354]:
#Initial values
f = np.array([0.5,1.15,1.3456,24.0,28])
h = 0.01
n = 10000

#define subplot
fig, axs = plt.subplots(1, 5, figsize=(15, 4), sharex=False,sharey=False)

#loop to create subplots of lorenz-attractor for different values of r
for i in range(5):
    r = f[i]
    x,y,z = lorenz_solver(r,h,n)
    axs[i].plot(x, y)
    axs[i].set_title("r ={}".format(r))
    print(i)
plt.savefig("lorenz.png")

finished
0
finished
1
finished
2
finished
3
finished
4


Exercise 2.2

In [358]:
#Initial values
r = 27
h = 0.01
n = 5500
x,y,z = lorenz_solver(r,h,n)

#Filling an boolean array with true value at local minimum index
x = np.r_[True, z[1:] < z[:-1]] & np.r_[z[:-1] < z[1:], True]
x[0]= False
x[len(z)-1]= False

#Defining new arrays for z and step t
z_new = np.array([])
t_new = np.array([])

#Fill arrays with the local minimums
for i in range(len(x)):
    if x[i] == True: 
        z_new = np.append(z_new,z[i])
        t_new = np.append(t_new,t[i])
        
#Plot results
plt.plot(z_new[:-1],z_new[1:], ".", label="local minima $z_{k+1}=f(z_k)$")

#Fit linear function to the values of z_{k+1} as a function of z_k
def linear(x,m,b):
    return m*x+b

popt,pcov = curve_fit(linear,z_new[:-1],z_new[1:])

#Plot the resuluts
plt.plot(z_new[:-1],linear(z_new[:-1],*popt),color="black", label="Slope m ={}".format(np.round(popt[0],4)))

plt.legend()
plt.title("r ={}, h={}, n={}".format(r,h,n))
plt.xlabel("$z_k$")
plt.ylabel("$z_{k+1}=f(z_k)$")
plt.savefig("lorenzz.png")

finished


GIF-animation

In [357]:
#Initial values
r = 27
h = 0.01
n = 10000
x,y,z = lorenz_solver(r,h,n)

#set 3D plot
fig = plt.figure()
ax = Axes3D(fig)

#define update function for animation
def update(num, data, line, obj):
    i = int(num*n/N)
    line.set_data(data[:2, :i])
    obj.set_data(data[:2, i])
    line.set_3d_properties(data[2, :i])
    obj.set_3d_properties(data[2, i])
    ax.view_init(30, num*0.2)

#set values for axes
N = 1000
data = np.array(x_vec).T
line, = ax.plot(data[0, 0:1], data[1, 0:1], data[2, 0:1], color="black")
obj, = ax.plot(data[0, 0:1], data[1, 0:1], data[2, 0:1],marker="o",color="#21a9a0")

# Setting the axes properties
ax.set_xlim3d([np.min(x)*1.1, np.max(x)*1.1])
ax.set_xlabel('X')

ax.set_ylim3d([np.min(y)*1.1, np.max(y)*1.1])
ax.set_ylabel('Y')

ax.set_zlim3d([np.min(z)*1.1, np.max(z)*1.1])
ax.set_zlabel('Z')

#animate and save
ani = animation.FuncAnimation(fig, update, fargs=(data, line, obj), interval=1, blit=False, frames=N)
#ani.save('lorenz.gif', writer='pillow', fps=50)

finished
